##### 출처 : https://devkyu.tistory.com/907

In [1]:
import requests
#import time
import pandas as pd
 
from bs4 import BeautifulSoup as bs
from datetime import datetime

In [2]:
def exportData(code, startdate, enddate):
    
    isEnd = False
    res = {
        'index': [],
        'data': []
    }

    print("Start")
    
    page = 1
    while isEnd != True:
        url = f'https://finance.naver.com/item/sise_day.nhn?code={code}&page={page}'
        response = requests.get(url)
        html = bs(response.text, 'html.parser')
        
        # parse
        trList = html.find_all("tr", {"onmouseover":"mouseOver(this)"})
        for tr in trList:
            tdList = tr.find_all('td')
            
            date = tdList[0].text.strip()  # 날짜
            close_price = int(tdList[1].text.strip().replace(',', ''))  # 종가
            diff_price = int(tdList[2].text.strip().replace(',', ''))  # 전일비
            open_price = int(tdList[3].text.strip().replace(',', ''))  # 시가
            high_price = int(tdList[4].text.strip().replace(',', ''))  # 고가
            low_price = int(tdList[5].text.strip().replace(',', ''))  # 저가
            volume = int(tdList[6].text.strip().replace(',', ''))  # 거래량
            
            target_date = datetime.fromisoformat(date.replace('.', '-'))
            if target_date < start:
                isEnd = True
                break
            elif target_date < end and target_date > start:
                #print(target_date)
                # insert index
                res['index'].insert(0, target_date)
                
                # insert data with ["Open","High","Low","Close","Volume"]
                res['data'].insert(0, [open_price, high_price, low_price, close_price,\
                                       volume])
                        
        page += 1
        #time.sleep(2)
        
    df = pd.DataFrame(data=res['data'], index=res['index'],\
                      columns=["Open","High","Low","Close","Volume"])
    #df.to_json(code+'.json', orient='split', date_format='index')
    
    print("Start - Done")
    
    return df

In [3]:
code = "251350"
start = datetime(2020, 1, 1)
end = datetime(2020, 10, 1)
 
result = exportData(code, start, end)

Start
Start - Done


In [4]:
import FinanceDataReader as fdr

df = fdr.DataReader('251350')
df = df.add_suffix('_2')

In [5]:
result_con = pd.concat([result, df], axis=1)
result_Close = result_con.loc[result_con.index >= start, ['Close', 'Close_2']]
result_Close.loc[result_Close['Close'] != result_Close['Close_2']]

,Close,Close_2


In [6]:
result_Close

,Close,Close_2
2020-01-02,15140.0,15140
2020-01-03,15175.0,15175
2020-01-06,15205.0,15205
2020-01-07,15265.0,15265
2020-01-08,15245.0,15245
...,...,...
2020-09-23,15000.0,15000
2020-09-24,14725.0,14725
2020-09-25,14875.0,14875
2020-09-28,15200.0,15200
